In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
import math
import warnings
warnings.filterwarnings("ignore")

In [9]:
with open('data/R2_STD_IVS_DFW.pkl', 'rb') as file:
    standard_ivs_params = pickle.load(file)

In [14]:
for i in standard_ivs_params:
    print(standard_ivs_params[i])
    break

('2010-02-08',               0.80      0.84      0.88      0.92      0.96      1.00  \
0.041096  0.378948  0.347878  0.320059  0.295491  0.274174  0.256107   
0.082192  0.374848  0.343998  0.316400  0.292052  0.270955  0.253109   
0.164384  0.366870  0.336462  0.309304  0.285397  0.264741  0.247335   
0.249315  0.358939  0.328986  0.302284  0.278832  0.258631  0.241682   
0.334247  0.351326  0.321828  0.295581  0.272585  0.252840  0.236345   
0.416438  0.344260  0.315203  0.289397  0.266842  0.247538  0.231484   
0.498630  0.337492  0.308876  0.283511  0.261396  0.242533  0.226920   
0.747945  0.318782  0.291503  0.267475  0.246697  0.229171  0.214895   
1.000000  0.302648  0.276720  0.254044  0.234618  0.218444  0.205520   
1.498630  0.278970  0.255717  0.235715  0.218964  0.205463  0.195213   
2.000000  0.266201  0.245636  0.228323  0.214260  0.203448  0.195887   

              1.04      1.08      1.12      1.16      1.20  
0.041096  0.241291  0.229726  0.221412  0.216348  0.214536 

## Test for Calender Arbitrage

In [5]:
break_counter = 0
for date in standard_ivs_params:
    test_df = standard_ivs_params[date]
    df_t_deriv = test_df.diff(periods=1, axis=0).fillna(method='bfill')
    exit_loop = False
    for moneyness in test_df.columns:
        for dte in test_df.index:
            iv = test_df.loc[dte].loc[moneyness]
            pderiv_t = df_t_deriv.loc[dte].loc[moneyness]
            l_cal = iv + (2 * (dte * 1.0) * pderiv_t)
            if l_cal < 0:
                exit_loop = True
                break
        if exit_loop == True:
            break_counter += 1
            break
break_counter

77

## Test for Butterfly Arbitrage

In [5]:
break_counter = 0
for date in standard_ivs_params:
    test_df = standard_ivs_params[date][0]
    df_m_deriv = test_df.diff(periods=1, axis=1).fillna(method='bfill', axis=1)
    df_mm_deriv = test_df.diff(periods=1, axis=1).fillna(method='bfill', axis=1).diff(periods=1, axis=1).replace(0.0, np.nan).fillna(method='bfill', axis=1)
    exit_loop = False
    for moneyness in test_df.columns:
        for dte in test_df.index:
            pderiv_m = df_m_deriv.loc[dte].loc[moneyness]
            pderiv_mm = df_mm_deriv.loc[dte].loc[moneyness]
            iv = test_df.loc[dte].loc[moneyness]
            l_but = ((1 - ((math.log(moneyness) * pderiv_m) / (iv))) ** 2) - (((iv * dte * pderiv_m) ** 2) / 4.0) + (dte * iv * pderiv_mm)
            if l_but < 0:
                exit_loop = True
                break
        if exit_loop == True:
            break_counter += 1
            break
break_counter

0

## Non-negative derivate test

In [6]:
count = 0
for date in standard_ivs_params:
    surface = standard_ivs_params[date][0]
    surface = surface ** 2
    ivars = surface.multiply(surface.index.to_series(), axis=0)
    minimum = ivars.diff(periods=1, axis=0).min().min()
    if minimum < 0:
        count += 1
count

361